In [1]:
#Importing all the librarie
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline

In [2]:
mails = pd.read_csv('spam.csv', encoding = 'latin-1')          #loading the data into mails
mails.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)  mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True) #dropping the unknown columns from the data
mails.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
mails.rename(columns = {'v1': 'labels', 'v2': 'message'}, inplace = True)     #renaming the columns of the data
mails.head()

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
mails['labels'].value_counts()              #counting the total number of ham and spam mails

ham     4825
spam     747
Name: labels, dtype: int64

In [7]:
mails['label'] = mails['labels'].map({'ham': 0, 'spam': 1})              #spam:1, ham:0
mails.head()

,labels,message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
mails.drop(['labels'], axis = 1, inplace = True)                    # dropping the column having name of the classes of mails 
mails.head()

,message,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [9]:
totalMails = 4825 + 747
trainIndex, testIndex = list(), list()                      #splitting the training and testing data
for i in range(mails.shape[0]):
    if np.random.uniform(0, 1) < 0.70:
        trainIndex += [i]
    else:
        testIndex += [i]
trainData = mails.loc[trainIndex]
testData = mails.loc[testIndex]

In [10]:
trainData.reset_index(inplace = True)
trainData.drop(['index'], axis = 1, inplace = True)
trainData.head()

,message,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Free entry in 2 a wkly comp to win FA Cup fina...,1
2,U dun say so early hor... U c already then say...,0
3,"Nah I don't think he goes to usf, he lives aro...",0
4,FreeMsg Hey there darling it's been 3 week's n...,1


In [11]:
trainData['label'].value_counts()                 #coutning no. of spam and ham emails in training data

0    3402
1     502
Name: label, dtype: int64

In [12]:
testData['label'].value_counts()                  #coutning no. of spam and ham emails in test data

0    1423
1     245
Name: label, dtype: int64

In [15]:
trainData.head()              #training data

,message,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Free entry in 2 a wkly comp to win FA Cup fina...,1
2,U dun say so early hor... U c already then say...,0
3,"Nah I don't think he goes to usf, he lives aro...",0
4,FreeMsg Hey there darling it's been 3 week's n...,1


In [17]:
testData.head() #test data

,message,label
1,Ok lar... Joking wif u oni...,0
6,Even my brother is not like to speak with me. ...,0
11,"SIX chances to win CASH! From 100 to 20,000 po...",1
12,URGENT! You have won a 1 week FREE membership ...,1
23,Aft i finish my lunch then i go str down lor. ...,0


In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
cv1=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector1=cv1.fit_transform(trainData['message'])  #word count vector for train data
#word_count_vector2=cv.fit_transform(testData['message'])  #word count vector for test data
print(word_count_vector1.shape)     # shape of word count vector for train data
#print(word_count_vector2.shape)     #  shape of word count vector for test data

(3904, 7197)


In [21]:
tfidf_transformer1=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer1.fit(word_count_vector1)
# count matrix for train data
count_vector1=cv1.transform(trainData['message'])
# count matrix for test data
#count_vector2=cv.transform(testData['message']) 

# tf-idf scores for train and test data
tf_idf_vector1=tfidf_transformer1.transform(count_vector1)
#tf_idf_vector2=tfidf_transformer.transform(count_vector2)

feature_names1 = cv1.get_feature_names()
 
#get tfidf vector for first document
first_document_vector1=tf_idf_vector1[0]
 
#print the scores
df1 = pd.DataFrame(first_document_vector1.T.todense(), index=feature_names1, columns=["tfidf_train"])
df1.sort_values(by=["tfidf_train"],ascending=False)

train_tdifval = df1["tfidf_train"].tolist()
#train_tdifval = np.array(train_tdifval)
train_tdifval = list(train_tdifval)

train_tdifval1 = []
for i in train_tdifval:
    train_tdifval1.append([i])

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
cv2=CountVectorizer()
 
# this steps generates word counts for the words in your docs
#word_count_vector1=cv1.fit_transform(trainData['message'])  #word count vector for train data
word_count_vector2=cv2.fit_transform(testData['message'])  #word count vector for test data
#print(word_count_vector1.shape)     # shape of word count vector for train data
print(word_count_vector2.shape)     #  shape of word count vector for test data

(1668, 4481)


In [23]:
tfidf_transformer2=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer2.fit(word_count_vector2)
# count matrix for train data
#count_vector1=cv1.transform(trainData['message'])
# count matrix for test data
count_vector2=cv2.transform(testData['message']) 

# tf-idf scores for train and test data
#tf_idf_vector1=tfidf_transformer1.transform(count_vector1)
tf_idf_vector2=tfidf_transformer2.transform(count_vector2)

feature_names2 = cv2.get_feature_names()
 
#get tfidf vector for first document
first_document_vector2=tf_idf_vector2[0]
 
#print the scores
df2 = pd.DataFrame(first_document_vector2.T.todense(), index=feature_names2, columns=["tfidf_test"])
df2.sort_values(by=["tfidf_test"],ascending=False)
#test_vector = df2["tfidf_test"]
#print(test_vector)
test_tdifval = df2["tfidf_test"].tolist()
#test_tdifval = np.array(test_tdifval)
test_tdifval = list(test_tdifval)

test_tdifval1 = []
for i in test_tdifval:
    test_tdifval1.append([i])

In [24]:
def distance1(instance1, instance2):     #function for euclidean distance
    findist = 0
    for i1 in instance1:
        for i2 in i1:
            for j1 in instance2:
                for j2 in [j1]:
                    dist = (i2-j2)**2
                    findist = findist+dist
    final_dist = findist**(1/2) 
    return final_dist

In [25]:
#function for finding the neighbors
import operator
def get_neighbors(train_vector, test_vector_instance, k, distance=distance1):
    distances = []
    for x in range(len(train_vector)):        
        dist =  distance1(test_vector_instance, train_vector[x])
        #dist = euclidean_distance(train_vector[x], test_vector_instance, norm)
        distances.append((train_vector[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


In [26]:
#function to get the prediction
from collections import Counter
def get_response(neighbors):
    class_counter = Counter()
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        class_counter[response] += 1        
        return class_counter.most_common(1)[0][0]

In [27]:
#function for finding the testing accuracy of the classifier
def accuracy_NN(test_vector, prediction):
    correct = 0
    for x in range(len(test_vector)):
        if test_vector[x][-1] == prediction[x]:
            correct += 1
    return ((correct / float(len(test_vector))) * 100.0)

In [ ]:
prediction = []
k = 5
for x in range(len(test_tdifval1)):
    neighbors = get_neighbors(train_tdifval1, test_tdifval1, 5, distance1)
    result = get_response(neighbors)
    prediction.append(result)
accuracy = accuracy_NN(test_tdifval1, prediction)
print('The accuarcy for '+ str(k) + '-NN is ' + str(accuracy))